In [27]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler, MinMaxScaler, maxabs_scale, MaxAbsScaler
import time
import random

In [28]:
def balance_data(df):
    # *************************
    ##Create a balanced Data
    # *************************
    for i in range(1, 8):
        print(i)
        #         print(df.index)
        cond = df.Cover_Type == i
        subset = df[cond].dropna()
        #         print(subset)
        if i == 4:
            subset = subset.sample(n=2747)
        else:
            subset = subset.sample(n=2747)
        if i == 1:
            balanced_data = subset
        else:
            balanced_data = balanced_data._append(subset)
            # frames = [balanced_data, subset]
 
            # balanced_data = pd.concat(frames)
    return balanced_data

In [29]:
def getData(filename, balance=False):
    chunksize = 1200000
    flag =1
    for data in pd.read_csv(filename, sep=",", chunksize=chunksize):
        while flag<2 and chunksize < 1200000:
            print(data)
        flag+=1
#     print(data.head())
    print(list(data.columns.values))
    print(type(data))
    if balance:
        data = balance_data(data)
    return data

In [30]:
filename = '../Dataset/covtype.csv'
dataset = getData(filename)
print(dataset)

['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', 'Cover_Type']
<class 'pandas.core.frame.DataFrame'>
        Elevation  Aspect  Slope  Horizontal_Dist

# drop HI9

In [31]:
dataset = dataset.drop("Hillshade_9am", axis=1) # delete HI9


In [32]:
dataset = dataset.drop(labels=range(581012,581016 ), axis=0)

# whole data normalized

In [75]:
def normalize_data(rawData):
    # training
    norm_tcolumns = rawData[rawData.columns[:9]]  # only the first nine columns need normalization, the rest is binary
    #     scaler = MinMaxScaler(copy=True, feature_range=(0, 1)).fit(norm_tcolumns.values)
    scaler = MaxAbsScaler(copy=True).fit(norm_tcolumns.values)
    scaledf = scaler.transform(norm_tcolumns.values)
    training_examples = pd.DataFrame(scaledf, index=norm_tcolumns.index,
                                     columns=norm_tcolumns.columns)  # scaledf is converted from array to dataframe
    rawData.update(training_examples)

    return rawData

In [76]:
normalized_dataset=normalize_data(dataset)

In [79]:
normalized_dataset.to_csv('Cover_type_wholeData_normalized.csv', index=False)

# reverse one-hot encoding (for soil type)

In [37]:
#split the encoded data's for soil type
dataset_soilType=dataset.iloc[:,13:53] # from 14th to 54 column
print(dataset_soilType)

        Soil_Type1  Soil_Type2  Soil_Type3  Soil_Type4  Soil_Type5  \
0              0.0         0.0         0.0         0.0         0.0   
1              0.0         0.0         0.0         0.0         0.0   
2              0.0         0.0         0.0         0.0         0.0   
3              0.0         0.0         0.0         0.0         0.0   
4              0.0         0.0         0.0         0.0         0.0   
...            ...         ...         ...         ...         ...   
581007         0.0         1.0         0.0         0.0         0.0   
581008         0.0         1.0         0.0         0.0         0.0   
581009         0.0         1.0         0.0         0.0         0.0   
581010         0.0         1.0         0.0         0.0         0.0   
581011         0.0         1.0         0.0         0.0         0.0   

        Soil_Type6  Soil_Type7  Soil_Type8  Soil_Type9  Soil_Type10  ...  \
0              0.0         0.0         0.0         0.0          0.0  ...   
1      

In [44]:
dataset_2= dataset # creating new dataset for soil-type
dataset_soilType_list=dataset_soilType.values.tolist()


In [47]:
len(dataset_soilType_list)

581012

In [54]:
#insert new column "Soil_type" into dataset
SoilType=[]
for i in range(0,len(dataset_soilType_list)):
    for j in range(0,len(dataset_soilType_list[0])):
        if dataset_soilType_list[i][j]==1:
            SoilType.append(j+1)
# SoilType

dataset_2.insert( 12, "Soil_type",SoilType, True)

In [58]:
dataset_2.to_csv('Cover_type_short_version.csv', index=False)

# reverse one-hot encoding (for wilderness)

In [68]:
# dataset = dataset.drop("Soil_type", axis=1)

In [69]:
dataset

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596.0,51.0,3.0,258.0,0.0,510.0,232.0,148.0,6279.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
1,2590.0,56.0,2.0,212.0,-6.0,390.0,235.0,151.0,6225.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2804.0,139.0,9.0,268.0,65.0,3180.0,238.0,135.0,6121.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,122.0,6211.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,2595.0,45.0,2.0,153.0,-1.0,391.0,234.0,150.0,6172.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396.0,153.0,20.0,85.0,17.0,108.0,237.0,118.0,837.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
581008,2391.0,152.0,19.0,67.0,12.0,95.0,237.0,119.0,845.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
581009,2386.0,159.0,17.0,60.0,7.0,90.0,241.0,130.0,854.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
581010,2384.0,170.0,15.0,60.0,5.0,90.0,245.0,143.0,864.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [70]:
#split the encoded data's for soil type
dataset_Wilderness=dataset.iloc[:,9:13] # from 14th to 54 column
print(dataset_Wilderness)

        Wilderness_Area1  Wilderness_Area2  Wilderness_Area3  Wilderness_Area4
0                    1.0               0.0               0.0               0.0
1                    1.0               0.0               0.0               0.0
2                    1.0               0.0               0.0               0.0
3                    1.0               0.0               0.0               0.0
4                    1.0               0.0               0.0               0.0
...                  ...               ...               ...               ...
581007               0.0               0.0               1.0               0.0
581008               0.0               0.0               1.0               0.0
581009               0.0               0.0               1.0               0.0
581010               0.0               0.0               1.0               0.0
581011               0.0               0.0               1.0               0.0

[581012 rows x 4 columns]


In [71]:
dataset_3= dataset_2 # creating new dataset for soil-type
dataset_Wilderness_list=dataset_Wilderness.values.tolist()


In [72]:
#insert new column "Soil_type" into dataset
Wilderness=[]
for i in range(0,len(dataset_Wilderness_list)):
    for j in range(0,len(dataset_Wilderness_list[0])):
        if dataset_Wilderness_list[i][j]==1:
            Wilderness.append(j+1)
# SoilType

dataset_3.insert( 9, "Wilderness",Wilderness, True)

In [73]:
dataset_3

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596.0,51.0,3.0,258.0,0.0,510.0,232.0,148.0,6279.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
1,2590.0,56.0,2.0,212.0,-6.0,390.0,235.0,151.0,6225.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2804.0,139.0,9.0,268.0,65.0,3180.0,238.0,135.0,6121.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,122.0,6211.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,2595.0,45.0,2.0,153.0,-1.0,391.0,234.0,150.0,6172.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396.0,153.0,20.0,85.0,17.0,108.0,237.0,118.0,837.0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
581008,2391.0,152.0,19.0,67.0,12.0,95.0,237.0,119.0,845.0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
581009,2386.0,159.0,17.0,60.0,7.0,90.0,241.0,130.0,854.0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
581010,2384.0,170.0,15.0,60.0,5.0,90.0,245.0,143.0,864.0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [74]:
dataset_3.to_csv('Cover_type_super_short_version.csv', index=False)

# super short dataset normalized

In [81]:
dataset_4=df = pd.read_csv('../shot_listed_dataset/Cover_type_super_short_version.csv')

In [82]:
dataset_4

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness,Soil_type,Cover_Type
0,2596,51,3,258,0,510,232,148,6279,1,29,5
1,2590,56,2,212,-6,390,235,151,6225,1,29,5
2,2804,139,9,268,65,3180,238,135,6121,1,12,2
3,2785,155,18,242,118,3090,238,122,6211,1,30,2
4,2595,45,2,153,-1,391,234,150,6172,1,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,237,118,837,3,2,3
581008,2391,152,19,67,12,95,237,119,845,3,2,3
581009,2386,159,17,60,7,90,241,130,854,3,2,3
581010,2384,170,15,60,5,90,245,143,864,3,2,3


In [86]:
def normalize_data(rawData):
    # training
    norm_tcolumns = rawData[rawData.columns[0:11]]  # only the first nine columns need normalization, the rest is binary
    #     scaler = MinMaxScaler(copy=True, feature_range=(0, 1)).fit(norm_tcolumns.values)
    scaler = MaxAbsScaler(copy=True).fit(norm_tcolumns.values)
    scaledf = scaler.transform(norm_tcolumns.values)
    training_examples = pd.DataFrame(scaledf, index=norm_tcolumns.index,
                                     columns=norm_tcolumns.columns)  # scaledf is converted from array to dataframe
    rawData.update(training_examples)

    return rawData

In [87]:
super_short_dataset_normalized=normalize_data(dataset_4)
super_short_dataset_normalized

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness,Soil_type,Cover_Type
0,0.672888,0.141667,0.045455,0.184681,0.000000,0.071659,0.913386,0.582677,0.875366,0.25,0.725,5
1,0.671332,0.155556,0.030303,0.151754,-0.009983,0.054798,0.925197,0.594488,0.867838,0.25,0.725,5
2,0.726801,0.386111,0.136364,0.191840,0.108153,0.446817,0.937008,0.531496,0.853339,0.25,0.300,2
3,0.721877,0.430556,0.272727,0.173228,0.196339,0.434172,0.937008,0.480315,0.865886,0.25,0.750,2
4,0.672628,0.125000,0.030303,0.109520,-0.001664,0.054939,0.921260,0.590551,0.860449,0.25,0.725,5
...,...,...,...,...,...,...,...,...,...,...,...,...
581007,0.621047,0.425000,0.303030,0.060845,0.028286,0.015175,0.933071,0.464567,0.116688,0.75,0.050,3
581008,0.619751,0.422222,0.287879,0.047960,0.019967,0.013348,0.933071,0.468504,0.117803,0.75,0.050,3
581009,0.618455,0.441667,0.257576,0.042949,0.011647,0.012646,0.948819,0.511811,0.119058,0.75,0.050,3
581010,0.617937,0.472222,0.227273,0.042949,0.008319,0.012646,0.964567,0.562992,0.120452,0.75,0.050,3


In [88]:
super_short_dataset_normalized.to_csv('Cover_type_super_short_version_normalized.csv', index=False)